## Bakery Example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vitostamatti/mathematical-optimization-pyomo/blob/main/notebooks/01-bakery.ipynb)


A local bakery is planning the elaboration of five diferent products (P1, P2, P3, P4 and P5)
for two types of clients (final consumers and resellers). The master baker knows that the 
correct usage of the ovens is critic and wishes to plan all the work for the next week.

Currently, the bakery has 4 ovens (H1, H2, H3 and H4) that work in parallel. Each one
has a specific capacity which is defined by the number of trays that can be allocated inside.
The costs of using each oven also varies according with its size.


| Oven    | Capacity (trays) | Costs (USD/hs)|
| ------- | ---------------- | ------------- |
| O1      | 10               | 20            |
| O2      | 12               | 40            |
| O3      | 9                | 25            |
| O4      | 7                | 50            |


The processing time of each product depends on the specific product and the
chosen oven. The next table shows the time in minutes required to cook
on batch of each product in each oven. Cells with a 'X' means that 
this particular combination of Product-Oven is not alowed.
The baker works 4 hours a day from de Monday to Saturday.


|Product |O1       |O2      |O3     |O4     |
|--------|---------|--------|-------|-------|
|P1      |30       |25      |X      |15     |
|P2      |40       |35      |X      |20     |
|P3      |30       |25      |40     |X      |
|P4      |50       |45      |40     |X      |
|P5      |60       |55      |40     |15     |



Ovens are considered to operate, without exception, at their maximum load capacity.
Because processing times are different for each product type,
each oven can cook only one product at a time. In addition to the cooking time it takes
each batch, a loading time of the trays inside the oven must be considered.


|Prod |Trays Capacity (kg/tray)          |Load Time (min/tray)          |
|-----|----------------------------------|------------------------------|
|P1   |4.0                               |3.0                           |
|P2   |3.0                               |2.5                           |
|P3   |5.0                               |2.0                           |
|P4   |3.5                               |4.0                           |
|P5   |2.5                               |1.5                           |


The production must satisfy both classes of demand. Each type has its
own sell price as well as an estimated amount per week.


|Prod | Reseller Dem. (kg)       | Reseller Sell Price ($/kg)   | 
|-----|--------------------------|------------------------------|
|P1   |510                       |20                            |
|P2   |550                       |18                            |
|P3   |600                       |25                            |
|P4   |615                       |15                            |
|P5   |470                       |30                            |


For final customers the bakery has a specific final sale price. In function of
sales statistics for this type of customer is established, on the one hand,
a minimum demand that must be respected, and on the other, an objective demand or "target"
of company customers. However, this "target" can be exceeded if it is
convenient, since there is the possibility of placing the surplus on the market at a
lower price (50% of the final sale price), aimed at promoting the product and
attract customers from the competition.


|Prod |Final Consumer Min. Dem.(kg)|Final Consumer Target Dem.(kg)|Sell Price Final Consumer($/kg)|
|-----|----------------------------|------------------------------|-------------------------------|
|P1   |124                         |248                           |25                             |
|P2   |100                         |200                           |23                             |
|P3   |120                         |240                           |31                             |
|P4   |130                         |252                           |16                             |
|P5   |110                         |216                           |32                             |


In addition, it should be considered that if the production of any product is below the "target",
lost sales generate a cost of 20% of the sale price due to the loss of
future sales in the hands of the competition.

In addition, the planner knows some limitations of the productive system,
which should be taken into account when developing the program. Namely:

1. Products P1 and P2 cannot be produced in the same oven.
2. None of the ovens can produce more than two types of products.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
import pandas as pd

import shutil
import sys
import os.path


try:
    import pyomo.environ as pyo
except:
    !pip install pyomo
    !conda install -c conda-forge glpk


if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc
        except:
            pass

# assert(shutil.which("cbc") or os.path.isfile("cbc"))


### Sets Declaration

In [52]:
#SETS
products = ['P' + str(i) for i in range(1, 6)] 
ovens = ['O'+ str(i) for i in range(1,5)]
print(products)
print(ovens)

['P1', 'P2', 'P3', 'P4', 'P5']
['O1', 'O2', 'O3', 'O4']


In [53]:
o_p = {
    'O1': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'O2': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'O3': ['P3', 'P4', 'P5'],
    'O4': ['P1', 'P2', 'P5'],
}

In [54]:
# demand
dem = {
    'P1': 510,
    'P2': 550,
    'P3': 600,
    'P4': 615,
    'P5': 470,
}

In [55]:
#sell price to reseller
sp_reseller = {
    'P1': 20,
    'P2': 18,
    'P3': 25,
    'P4': 15,
    'P5': 30,
}


In [56]:
#sell price to final consumer
sp_consumer = {
    'P1': 25,
    'P2': 23,
    'P3': 31,
    'P4': 16,
    'P5': 32,
}

In [57]:
#dsmax(p) max demand
dmax = {
    'P1': 248,
    'P2': 200,
    'P3': 240,
    'P4': 252,
    'P5': 216,
}


In [58]:
# min demand
dmin = {
    'P1': 124,
    'P2': 100,
    'P3': 120,
    'P4': 130,
    'P5': 110,
}



# oven capacity in trays
capacity = {
    'O1': 10,
    'O2': 12,
    'O3': 9,
    'O4': 7,
}



# kg in each tray
kg_tray = {
    'P1': 4.0,
    'P2': 3.0,
    'P3': 5.0,
    'P4': 3.5,
    'P5': 2.5,
}


# production cost (USD/hs)
pc = {
    'O1': 20,
    'O2': 40,
    'O3': 25,
    'O4': 50,
}


# setup time (min)
setup = {
    'P1': 3,
    'P2': 2.5,
    'P3': 2,
    'P4': 4,
    'P5': 1.5,
}


In [64]:
# production capacity in kg
prod_cap = {
    (o,p): kg_tray[p]*capacity[o] for o in ovens for p in o_p[o]
    }


In [65]:
#pv3 precio de venta por encima de la demanda maxima
sp_over_max_dem = {p: 0.5*sp_consumer[p] for p in products}

#cvp costo de ventas perdidas
cost_lost_sell = { p: 0.2*sp_consumer[p] for p in products}

In [66]:
#dispt horas disponibles  /24/
working_hours = 24

In [67]:
# production time (min per tray)
prod_time = {
    ("P1","O1"): 30,
    ("P1","O2"): 25,
    ("P2","O1"): 40,
    ("P2","O2"): 35,
    ("P2","O4"): 20,
    ("P3","O1"): 30,
    ("P3","O2"): 25,
    ("P3","O3"): 40,
    ("P4","O1"): 50,
    ("P4","O2"): 45,
    ("P4","O3"): 40,
    ("P5","O1"): 60,
    ("P5","O2"): 55,
    ("P5","O3"): 40,
    ("P5","O4"): 25,
}

## Create Model

In [71]:
def build_model():
    m = pyo.ConcreteModel()
    #SETS
    m.P = pyo.Set(initialize=products)
    m.O = pyo.Set(initialize=ovens)
    m.o_p = pyo.Set(initialize= m.O*m.P, filter=lambda m, o, p: p in o_p[o])

    #PARAMETERS (optional)
    m.dem = pyo.Param(m.P, initialize=dem, default=0)
    m.sp_reseller = pyo.Param(m.P, initialize=sp_reseller, default=0)
    m.sp_consumer = pyo.Param(m.P, initialize=sp_consumer, default=0)
    m.dmax = pyo.Param(m.P, initialize=dmax, default=0)
    m.dmin = pyo.Param(m.P, initialize=dmin, default=0)
    m.cap = pyo.Param(m.O, initialize=capacity, default=0)
    m.kg_tray = pyo.Param(m.P, initialize=kg_tray, default=0)
    m.prod_cost = pyo.Param(m.O, initialize=pc, default=0, name='production cost')
    m.setup = pyo.Param(m.P, initialize=setup, default=0)
    m.prod_cap = pyo.Param(m.o_p, initialize=prod_cap, default=0)
    m.sp_over_max_dem = pyo.Param(m.P, initialize=sp_over_max_dem, default=0)
    m.cost_lost_sell = pyo.Param(m.P, initialize=cost_lost_sell, default=0)
    m.working_hours = pyo.Param(initialize=working_hours)
    m.prod_time = pyo.Param(m.P,m.O, initialize=prod_time, default=0)

    #VARIABLES
    #kg a producir del producto p en o
    m.Q = pyo.Var(m.O, m.P, domain=pyo.NonNegativeReals)

    #total de kg producidos
    m.QQ = pyo.Var(m.P, domain=pyo.NonNegativeReals)

    #tiempo total de produccion de cada h
    m.TTP = pyo.Var(m.O, domain=pyo.NonNegativeReals)

    #porcentaje de utilizacion de cada h
    m.u = pyo.Var(m.O, domain=pyo.NonNegativeReals)

    #exc(p) kg de excedente a sucursales de p
    m.exc = pyo.Var(m.P, domain=pyo.NonNegativeReals)

    #Tprep(h)
    m.Tprep = pyo.Var(m.O, domain=pyo.NonNegativeReals)

    #XE(p) kg para estado
    m.XE = pyo.Var(m.P, domain=pyo.NonNegativeReals)

    #XS(p) kg para sucursal
    m.XS = pyo.Var(m.P, domain=pyo.NonNegativeReals)

    #US(p) kg insatisfechos a sucursales
    m.US = pyo.Var(m.P, domain=pyo.NonNegativeReals)

    #Integer Variable
    #horneadas a producir del producto p en h
    m.L = pyo.Var(m.O, m.P, domain=pyo.Integers)
    m.L.ub=1000

    #Binary Variables
    # decision de producir el producto p en h
    m.y = pyo.Var(m.O, m.P, domain=pyo.Binary)


    #CONSTRAINTS
    def R1(m, o, p):
        return  m.Q[o,p] == m.prod_cap[o,p]*m.L[o,p] 
    m.R1 = pyo.Constraint(m.o_p, rule = R1) 

    def R2(m, o, p):
        return  m.L[o,p] >= m.y[o,p]
    m.R2 = pyo.Constraint(m.o_p, rule = R2) 

    def R3(m, o, p):
        return  m.L[o,p] <= 10000*m.y[o,p]
    m.R3 = pyo.Constraint(m.o_p, rule = R3) 

    def R4(m, p):
        return  m.QQ[p] == m.XE[p] + m.XS[p]
    m.R4 = pyo.Constraint(m.P, rule = R4)
        
    def R5(m, p):
        return  m.XE[p] == m.dem[p] 
    m.R5 = pyo.Constraint(m.P, rule = R5)

    def R6(m, p):
        return  m.XS[p] >= m.dmin[p] 
    m.R6 = pyo.Constraint(m.P, rule = R6)

    def R7(m, p):
        return  m.XS[p] == m.dmax[p] + m.exc[p] - m.US[p]
    m.R7 = pyo.Constraint(m.P, rule = R7)

    def R8(m, o):
        return  m.Tprep[o] + m.TTP[o] <= m.working_hours 
    m.R8 = pyo.Constraint(m.O, rule = R8)

    def R9(m, o):
        return m.y[o,'P1'] + m.y[o,'P2'] <= 1 
    m.R9 = pyo.Constraint(m.O, rule = R9)

    def R10(m, o):
        return sum(m.y[o,p] for p in m.P if (o,p) in m.o_p) <= 2
    m.R10 = pyo.Constraint(m.O, rule = R10)

    def R11(m, p):
        return sum(m.Q[o, p] for o in m.O if (o,p) in m.o_p) == m.QQ[p]
    m.R11 = pyo.Constraint(m.P, rule = R11)

    def R12(m, o):
        return m.TTP[o] == sum((m.prod_time[p, o]/60)*m.L[o, p] for p in m.P if (o,p) in m.o_p)     
    m.R12 = pyo.Constraint(m.O, rule = R12)

    def R13(m, o):
        return m.u[o] == 100*(m.TTP[o] + m.Tprep[o])/m.working_hours   
    m.R13 = pyo.Constraint(m.O, rule = R13)

    def R14(m, o):
        return m.Tprep[o] == sum((m.setup[p]/60)*m.L[o,p]*m.cap[o] for p in m.P if (o,p) in m.o_p )   
    m.R14 = pyo.Constraint(m.O, rule = R14)

    def OBJ(m):
        return (
            sum(
            m.sp_reseller[p]*m.XE[p]+
            m.sp_consumer[p]*(m.XS[p]-m.exc[p])+
            m.sp_over_max_dem[p]*m.exc[p]-
            m.cost_lost_sell[p]*m.US[p] 
            for p in m.P
            ) - 
            sum(
                m.TTP[o]*m.prod_cost[o] for o in m.O
                )
            )
        
    m.OBJ = pyo.Objective(rule = OBJ, sense = pyo.maximize)


    return m


m = build_model()

In [74]:
pyo.SolverFactory('glpk').solve(m).write() 
#SolverFactory('cbc').solve(m).write() 

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 78198.1333333333
  Upper bound: 78198.1333333333
  Number of objectives: 1
  Number of constraints: 98
  Number of variables: 88
  Number of nonzeros: 241
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 119
      Number of created subproblems: 119
  Error rc: 0
  Time: 0.05135488510131836
# ----------------------------------------------------------
#   Solution Information
# -------

In [77]:
for p in products:
    for o in ovens:
        if (o,p) in m.o_p and m.y[o,p].value>0:
            print(m.y[o,p], m.y[o,p].value)

y[O4,P1] 1.0
y[O1,P2] 1.0
y[O2,P3] 1.0
y[O2,P4] 1.0
y[O3,P4] 1.0
y[O3,P5] 1.0
y[O4,P5] 1.0


In [78]:
prod_plan = []
for var in m.Q.values():
    if var.value:
        if var.value>0:
            prod_plan.append([var.index()[0],var.index()[1],var.value])

In [79]:


df_prod_plan = pd.DataFrame(prod_plan, columns=['oven','product','kg'])

In [80]:
df_prod_plan

,oven,product,kg
0,O1,P2,660.0
1,O2,P3,840.0
2,O2,P4,336.0
3,O3,P4,409.5
4,O3,P5,180.0
5,O4,P1,784.0
6,O4,P5,420.0
